# TOPIC MODELLING

# LDA

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## About data
- It is consumer complaints dataset
- It is available in CFPB(Consumer Financial Protection Board) website.
- Rule of CFPB is to protect consumers from unfair,deceptive or abusive practices and take action against companies that break law.
- These companies are basically banking/financial entity.
- If consumers are not happy, or have been put aginst some charges and bank dosn't listen to them, then the consumers can go to CFPB, and CFPB addresses the grievances and help consumers solve the issue.

In [3]:
df=pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true', compression='zip', sep=',', quotechar='"')

In [4]:
df['Product'].value_counts()


Debt collection                21772
Credit card or prepaid card    13193
Mortgage                        9799
Checking or savings account     7003
Student loan                    2950
Vehicle loan or lease           2736
Name: Product, dtype: int64

In [5]:
df['Company'].value_counts() #most complaints are from CITIBANK

CITIBANK, N.A.                                                    3226
CAPITAL ONE FINANCIAL CORPORATION                                 2711
BANK OF AMERICA, NATIONAL ASSOCIATION                             2580
JPMORGAN CHASE & CO.                                              2409
WELLS FARGO & COMPANY                                             2001
                                                                  ... 
Time Investment Company, Inc.                                        1
Automotive Services Finance, Inc.                                    1
Foxstar Energy Resources LLC DBA Federal Student Loans Council       1
Uplift, Inc                                                          1
Keystone Credit Services LLC                                         1
Name: Company, Length: 2197, dtype: int64

In [6]:
## We will use complaint,product and company against which complaint is against for analysis
complaints_df=df[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'complaints'})

In [7]:
pd.set_option('display.max_colwidth', -1)
complaints_df.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,complaints,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION


## Train Test split (60:40)

In [8]:
X_train, X_hold = train_test_split(complaints_df, test_size=0.6, random_state=111)


In [9]:
stemmer = PorterStemmer()

In [10]:
def tokenize(text):
  # filtering out any words that has <3 characters(is,are,you etc..)
   tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2) ] 
   #stems = [stemmer.stem(item) for item in tokens]
   return tokens

## Vectorisation
- LDA requires word frequency data, so either use CountVectorizer or TfidfVectorizer withuse_idf=False and norm=None(so that we can use only TF part)
- max_df = 0.75 means frequency of a word can be max of 75%. Anything more than that would be considered a very common word
- min_df = 50 means atleast the word must be in 50 documents.If it is less than 50 documents then it can be a rare word

In [11]:
vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', max_df=0.75, min_df=50, max_features=10000, use_idf=False, norm=None)
tf_vectors = vectorizer_tf.fit_transform(X_train.complaints)  # using only complaints column

In [12]:
tf_vectors.shape

(22981, 2966)

In [34]:
vectorizer_tf.get_feature_names()# top 10000 features that it has picked

## Applying LDA
- LDA is a statistical model that allows a set of observation to be explained by a group or unobserved groups.
- In topic modelling it automatically classifies documents and estimates the relevance to various documents.
- n_components=6 means distribute it into 6 different topics. We had 6 products so we have put 6 . But this topic selection depends on domain knowledge.
- max_iter = 3 means we are runnig it for 3 iterations. Running it for higher iterations would give better results.
- Learning_method='online' is a more preferred option for large dataset.

In [14]:
lda = decomposition.LatentDirichletAllocation(n_components=6, max_iter=3, learning_method='online', learning_offset=50, n_jobs=-1, random_state=111)
W1 = lda.fit_transform(tf_vectors)
H1 = lda.components_

## Pick the top words of each and every topic
- What are the top words that are contributing to each topic, so that we can use the top words to identify, what is the customer talking about in each and every topic

In [16]:
num_words=15 #select top 15 words from each topic

vocab = np.array(vectorizer_tf.get_feature_names())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
topics #6 topics generated

['account payment credit card bank balance check late fees paid received told customer statement days',
 'mortgage loan home property escrow insurance payment modification request received date application servicing documents payments',
 'payment loan payments time paid told company month years loans months help make just money',
 'card bank credit account fraud charges chase dispute money charge america transaction claim fraudulent transactions',
 'debt credit account report collection company information reporting letter agency sent provide original received identity',
 'called told phone received number said asked information email sent letter company contact calls spoke']

## Interpretation
- Topic 1: About credit card , late fees
- Topic 2: Mortgages and property
- Topic 3: Loans
- Topic 5: About debt collection 
 - and so on 

In [18]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train.complaints))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1) #picking each prominant topic that document can belong using the LDA weights
df_doc_topic['dominant_topic'] = significant_topic

In [19]:
df_doc_topic


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.58,0.00,0.26,0.00,0.00,0.16,0
Doc1,0.89,0.10,0.00,0.00,0.00,0.00,0
Doc2,0.80,0.01,0.01,0.01,0.01,0.18,0
Doc3,0.00,0.00,0.00,0.00,0.38,0.61,5
Doc4,0.00,0.18,0.00,0.00,0.81,0.00,4
...,...,...,...,...,...,...,...
Doc22976,0.88,0.00,0.00,0.00,0.00,0.11,0
Doc22977,0.38,0.04,0.04,0.31,0.22,0.00,0
Doc22978,0.39,0.00,0.00,0.31,0.00,0.29,0
Doc22979,0.51,0.02,0.02,0.42,0.02,0.02,0


-  As you can compare X_train and df_doc_topic and see that 1st and 5th doc are rightly classified as Credit card and debt collection topics.

In [20]:
X_train.head()

,complaints,Product,Company
36524,"On XX/XX/2020. Checks for the full amounts owed were sent to each of these accounts from XXXX to Capital One : Capital One ending in # XXXX {$3700.00} Capital One ending in # XXXX {$680.00} Capital One ending in # XXXX {$2600.00} Capital One ending in # XXXX {$2100.00} Capital One ending in # XXXX {$3400.00} Total payment on these accounts : {$12000.00}. \nPrior to these payoff checks sent, I made my monthly payments due XX/XX/XXXX to avoid any late fees while these checks were received and processed by Capital One. \nI later received statements showing I owed interest on each one after payment in full was received. After sending them a letter priority mail on XX/XX/2020, asking why I was paying interest on top of interest, I received a letter on XX/XX/2020 letting me know the accounts were closed per my request, which I had requested they close since I was paying in full BUT no answer to my question. I attempted to call several times prior to mailing this letter but had not success in getting anyone on the line after waiting over an hour. I went ahead and paid the interest on these accounts which added to {$280.00}, out of fear I may be hit with a late fee. I attempted to call again XX/XX/2020. After waiting over an hour for a rep on phone, I was told I had to pay the interest on interest and also a residual fee after for each one. ( how do you pay interest on interest? ) Residual fee? On accounts paid in full? They said, that was their policy. \nShortly after making full payments on these accounts, On XX/XX/XXXX, my husband lost his job due to COVID and is currently unemployed. I have struggled to pay these additional fees that I find to be an illegal act by this credit card company. I was told if I did not pay the additional fees, a late fee will be added on top. I should not have to pay interest or residual fees on a payoff amount that had not outstanding balance. I also requested a copy of their agreement stating they are allowed to do this. I never received a response or any response to any of my questions. Again, we are in no financial position at this time due to loss of unemployment to be slammed with extra fees. It is unfortunate that the timing on these payments made in full and my husband losing his job came at a worse time.",Credit card or prepaid card,CAPITAL ONE FINANCIAL CORPORATION
46281,"On or about XX/XX/XXXX I opened a Citibank Basic Checking Account after seeing an offer described by Citibank that if I opened a Basic Checking Account, deposited at least {$5000.00} within 30 days of accounting opening, and held at least that amount in the account for 60 days that within 90 days after fulfilling the conditions of the agreement I would receive a {$200.00} checking account bonus. I soon deposited approximately {$5000.00} on XX/XX/XXXX into the account. On XX/XX/XXXX I received another email update from Citibank about the offer and thanking me for enrolling in the Citibank {$200.00} checking offer and repeating the terms of the agreement. \n\nOn or about XX/XX/XXXX I utilized the Citibank Mobile Application 's chat function to inquire as to when my bonus might be depositing, as it had been well over 60 days since when I opened the account and deposited the qualifying funds on XX/XX/XXXX. 60 days since my deposit would have been around early XX/XX/XXXX. The Mobile Application representative stated they could not resolve the issue and to call their One Stop Sales Unit, which I did on XX/XX/XXXX. The representative at the One Stop Sales Unit was confused and did not know how to search for the offer clearly, saying too that often the offers are for holding money in the account for 90 days or more, and provided essentially no clarity or guidance. \n\nI withdrew {$3000.00} of the {$5000.00} I had held in the account since XX/XX/XXXX on XX/XX/XXXX, my first withdrawl from the account. I am concerned as Citibank has been very unclear on when my deposit bonus will be depositing, as tha

## Let's test on Test data

In [21]:
WHold = lda.transform(vectorizer_tf.transform(X_hold.complaints[:5]))  #transform and generate weights on Test data


In [22]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_hold.complaints[:5]))]
df_doc_topic = pd.DataFrame(np.round(WHold, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [23]:
df_doc_topic


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.91,0.00,0.00,0.00,0.00,0.09,0
Doc1,0.11,0.51,0.00,0.00,0.00,0.36,1
Doc2,0.64,0.01,0.01,0.34,0.01,0.01,0
Doc3,0.00,0.06,0.81,0.00,0.00,0.13,2
Doc4,0.66,0.00,0.29,0.00,0.00,0.04,0


In [24]:
X_hold.head()


,complaints,Product,Company
30060,"I have a business checking account at BB & T. On XX/XX/2019, I attempted to deposit a check into my account and I received a message stating that I was over my monthly mobile deposit limit. I was confused because it was the first of the month and I had not deposited any checks since the previous month. I called BB & T and they said that I couldnt deposit checks into business accounts via the mobile app even though I had done that before. \n\nI was instructed to open a personal account, into which I could deposit checks via the mobile app. I was told that if I opened the account online I would have immediate access, that I could link my personal and business accounts, and immediately be able to transfer money between them. \n\nOn XX/XX/XXXX, I opened my personal account online. Though I successfully opened online, I did not have online access as I had been promised. Because I was traveling in an area where there were no BB & T branches, I could not go into a branch until XX/XX/2019. In the intervening time, my business account became overdrawn. I had the money in my personal account to bring it back into the black, but BB & T could not make the transfer until I went into a branch. During this time, I incurred an astonishing {$320.00} in overdraft fees in my business account because I had no way of transferring money between the two accounts. \n\nWhen I went into the branch, I met with XXXX XXXX in XXXX XXXX County, Florida. She investigated the accounts and told me she would link them and that I would have online access in XXXX hours. The online access still never material, I was not able to transfer money and I received an additional {$36.00} overdraft fee. This brought the total to {$360.00} in overdraft fees. \n\nI had only opened the second account on BB & Ts advice ; I could have had my customer send a bank wire directly into business account in the first week of XXXX to keep the account positive. My TOTAL deposits between the two accounts was always positive. \n\nI requested a refund and was told they would not refund any of the fees.",Checking or savings account,BB&T CORPORATION
53473,"To who it may concern, My concern is regarding Shellpoint Mortgage Servicing Company. This company has received my monthly mortgage payments and have failed to account it as payments received. My bank has sent me proof the account was cleared and paid. This company is trying to foreclosed on my house and has sent me a letter indicating they sent my file to their Loss Mitigation dept. The person who they assigned it to is XXXX XXXX XXXX x XXXX. Their corporate number is XXXX. \n\nI have can send proof of my claim from above should you need it. Please assist.",Mortgage,"Shellpoint Partners, LLC"
35879,I contacted XXXX about fraudulent charges that were made on my account and the customer service representative told me that they wouldnt be able to issue anew card or remove my fraudulent charges since the account was closed due to non-payment. I was unaware of this these charging that were made.. To my knowledge I didnt owe a payment because the account wasnt being used.,Credit card or prepaid card,"EQUIFAX, INC."
20993,"I first applied for the Fedloan Serving program in XXXX in hopes of getting loan forgiveness since I've been paying on my student loans since late XXXX or early XXXX and I have been a public servant since XXXX, XXXX. I have never missed a payment to any of the various loan companies that owned my loans. The Fedloan Program turned me down in XXXX, saying my loans, which were originally Federal Stafford Loans, did not qualify me for the Program. In XXXX, XXXX, I was solicited by XXXX XXXX and told I could get into this program to reduce my loan payments. I was charged {$690.00} by XXXX XXXX to get me into the Fedloan Serving program. My loan payments went down to $ XXXX/month for the first year. In XXXX, XXXX, Fedloan Servicing informed me via email that I needed to recertify my "" income-drive

## BERTOPIC
- Links: https://maartengr.github.io/BERTopic/algorithm/algorithm.html

In [25]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 3.5 MB/s 
     |████████████████████████████████| 88 kB 5.2 MB/s 
     |████████████████████████████████| 79 kB 9.0 MB/s 
     |████████████████████████████████| 5.2 MB 56.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.2 MB 30.4 MB/s 
     |████████████████████████████████| 1.2 MB 54.7 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
     |████████████████████████████████| 6.6 MB 48.6 MB/s 
     |████████████████████████████████| 636 kB 28.2 MB/s 
     |████████████████████████████████| 1.1 MB 60.2 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2342133 sha256=bb7d827ab4515db0ff8ed1419d0d6053b2716788885069b1edce46c37088c9a2
  Stored in directory: /root/.cache/p

In [26]:
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from bertopic import BERTopic

In [28]:
complaints_df.head()

,complaints,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never

In [35]:
# using same vectoriser as we used in LDA
vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', max_df=0.75, min_df=50, max_features=10000, use_idf=False, norm=None)
# create model 
model = BERTopic(verbose=True,top_n_words=15,vectorizer_model = vectorizer_tf)
 
#convert to list 
docs = complaints_df.complaints.to_list()
 
topics, probabilities = model.fit_transform(docs)

Batches:   0%|          | 0/1796 [00:00<?, ?it/s]

2022-06-16 08:28:09,047 - BERTopic - Transformed documents to Embeddings
2022-06-16 08:29:32,693 - BERTopic - Reduced dimensionality
2022-06-16 08:29:39,336 - BERTopic - Clustered reduced embeddings


In [36]:
model.get_topic_freq().head()

,Topic,Count
0,-1,26407
1,0,1566
2,1,1485
3,2,1084
4,3,844


In [65]:
tl = []
for j in range(10):
  l = []
  for i in model.get_topic(j):
    
    l.append(i[0])
  tl.append(l)
  

In [66]:
stl = []
for i in tl:
  st = ' '.join(i)
  stl.append(st)

In [67]:
stl

['capital card charge cards charges fraud merchant closed restricted late customer charged fraudulent security accounts',
 'executive branch overdraft checking check claim case banker money complaint home mortgage funds deposit transactions',
 'closing refinance officer rate lender loan close home process mortgage application lock disclosure costs refinancing',
 'hospital medical insurance collections visit patient billing covered bills billed agency collection provider emergency health',
 'loans student private repayment loan forbearance school principal rate afford payments paying lower income options',
 'modification trial foreclosure mortgage sale loan home appeal servicing servicer payments documents approved assistance paperwork',
 'mortgage late payments servicing check cashed loan month care grace transferred statement servicer make online',
 'discover card limit closed charges fraudulent score fraud form balance returns charge reason purchases used',
 'affidavit belong theft i

### Topics are regarding
- Mortgages
- student loans
- medical insurance
- credit card late fee

In [ ]:
## These were generated by LDA 
# ['account payment credit card bank balance check late fees paid received told customer statement days',
#  'mortgage loan home property escrow insurance payment modification request received date application servicing documents payments',
#  'payment loan payments time paid told company month years loans months help make just money',
#  'card bank credit account fraud charges chase dispute money charge america transaction claim fraudulent transactions',
#  'debt credit account report collection company information reporting letter agency sent provide original received identity',
#  'called told phone received number said asked information email sent letter company contact calls spoke']